# HAND GESTURE-BASED INTERACTION

---

Group members:
*   Ada Yılmaz
*   Ceren Şahin
*   Sima Adleyba
*   List item

### Installing necessary libraries and models

In [1]:
#install mediapipe
!pip install -q mediapipe

zsh:1: command not found: pip


In [2]:
#download a model that can recognize 7 hand gestures: 👍, 👎, ✌️, ☝️, ✊, 👋, 🤟
!wget -q https://storage.googleapis.com/mediapipe-models/gesture_recognizer/gesture_recognizer/float16/1/gesture_recognizer.task

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
#download test images from pixabay
import urllib

IMAGE_FILENAMES = ['thumbs_down.jpg', 'victory.jpg', 'thumbs_up.jpg', 'pointing_up.jpg']

for name in IMAGE_FILENAMES:
  url = f'https://storage.googleapis.com/mediapipe-tasks/gesture_recognizer/{name}'
  urllib.request.urlretrieve(url, name)

In [2]:
#or we can use our own images as shown below

# from google.colab import files
# uploaded = files.upload()

# for filename in uploaded:
#   content = uploaded[filename]
#   with open(filename, 'wb') as f:
#     f.write(content)
# IMAGE_FILENAMES = list(uploaded.keys())

# print('Uploaded files:', IMAGE_FILENAMES)

### Functions for visualization

In [3]:
#some functions to visualize the gesture recognition results.
import math
from matplotlib import pyplot as plt
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2

plt.rcParams.update({
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.spines.left': False,
    'axes.spines.bottom': False,
    'xtick.labelbottom': False,
    'xtick.bottom': False,
    'ytick.labelleft': False,
    'ytick.left': False,
    'xtick.labeltop': False,
    'xtick.top': False,
    'ytick.labelright': False,
    'ytick.right': False
})

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles


def display_one_image(image, title, subplot, titlesize=16):
    """Displays one image along with the predicted category name and score."""
    plt.subplot(*subplot)
    plt.imshow(image)
    if len(title) > 0:
        plt.title(title, fontsize=int(titlesize), color='black', fontdict={'verticalalignment':'center'}, pad=int(titlesize/1.5))
    return (subplot[0], subplot[1], subplot[2]+1)


def display_batch_of_images_with_gestures_and_hand_landmarks(images, results):
    """Displays a batch of images with the gesture category and its score along with the hand landmarks."""
    # Images and labels.
    images = [image.numpy_view() for image in images]
    gestures = [top_gesture for (top_gesture, _) in results]
    multi_hand_landmarks_list = [multi_hand_landmarks for (_, multi_hand_landmarks) in results]

    # Auto-squaring: this will drop data that does not fit into square or square-ish rectangle.
    rows = int(math.sqrt(len(images)))
    cols = len(images) // rows

    # Size and spacing.
    FIGSIZE = 13.0
    SPACING = 0.1
    subplot=(rows,cols, 1)
    if rows < cols:
        plt.figure(figsize=(FIGSIZE,FIGSIZE/cols*rows))
    else:
        plt.figure(figsize=(FIGSIZE/rows*cols,FIGSIZE))

    # Display gestures and hand landmarks.
    for i, (image, gestures) in enumerate(zip(images[:rows*cols], gestures[:rows*cols])):
        title = f"{gestures.category_name} ({gestures.score:.2f})"
        dynamic_titlesize = FIGSIZE*SPACING/max(rows,cols) * 40 + 3
        annotated_image = image.copy()

        for hand_landmarks in multi_hand_landmarks_list[i]:
          hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
          hand_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
          ])

          mp_drawing.draw_landmarks(
            annotated_image,
            hand_landmarks_proto,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())

        subplot = display_one_image(annotated_image, title, subplot, titlesize=dynamic_titlesize)

    # Layout.
    plt.tight_layout()
    plt.subplots_adjust(wspace=SPACING, hspace=SPACING)
    plt.show()

### Preview the images

In [4]:
import cv2
import math

DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480

def resize_and_show(image):
    h, w = image.shape[:2]
    if h < w:
        img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h / (w / DESIRED_WIDTH))))
    else:
        img = cv2.resize(image, (math.floor(w / (h / DESIRED_HEIGHT)), DESIRED_HEIGHT))
    
    # Display the image in a window with a name
    cv2.imshow('Resized Image', img)
    cv2.waitKey(0)  # Wait for a key press to close the window
    cv2.destroyAllWindows()  # Close the window after key press

# Example usage
images = {name: cv2.imread(name) for name in IMAGE_FILENAMES}

for name, image in images.items():
    if image is not None:
        print(f"Displaying: {name}")
        resize_and_show(image)
    else:
        print(f"Error: Could not read image {name}")


Displaying: thumbs_down.jpg


2024-12-07 15:15:04.982 Python[11988:558908] +[IMKClient subclass]: chose IMKClient_Legacy
2024-12-07 15:15:04.982 Python[11988:558908] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


Displaying: victory.jpg
Displaying: thumbs_up.jpg
Displaying: pointing_up.jpg


### Run the Google implementation
This first one is a how to from google, we can create our own by following the steps given.


In [5]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create an GestureRecognizer object.
base_options = python.BaseOptions(model_asset_path='gesture_recognizer.task')
options = vision.GestureRecognizerOptions(base_options=base_options)
recognizer = vision.GestureRecognizer.create_from_options(options)

images = []
results = []
for image_file_name in IMAGE_FILENAMES:
    
  # STEP 3: Load the input image.
  image = mp.Image.create_from_file(image_file_name)

  # STEP 4: Recognize gestures in the input image.
  recognition_result = recognizer.recognize(image)

  # STEP 5: Process the result. In this case, visualize it.
  images.append(image)
  top_gesture = recognition_result.gestures[0][0]
  hand_landmarks = recognition_result.hand_landmarks
  results.append((top_gesture, hand_landmarks))

display_batch_of_images_with_gestures_and_hand_landmarks(images, results)

I0000 00:00:1733573715.162574  558908 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1
W0000 00:00:1733573715.162988  558908 gesture_recognizer_graph.cc:129] Hand Gesture Recognizer contains CPU only ops. Sets HandGestureRecognizerGraph acceleration to Xnnpack.
I0000 00:00:1733573715.163858  558908 hand_gesture_recognizer_graph.cc:250] Custom gesture classifier is not defined.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1733573715.174755  560017 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733573715.183577  560017 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733573715.184255  560016 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback te

ValueError: Input image must contain three channel bgr data.

<Figure size 1300x1300 with 0 Axes>

### Example
Tried an example to see how it works and update the functions accordingly

In [10]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)

def detect_peace_sign(hand_landmarks):
    # Get coordinates of relevant landmarks
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]
    
    
    index_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
    middle_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]
    ring_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP]
    pinky_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP]


    # Define peace sign gesture: index and middle fingers up, ring and pinky fingers down
    if (
        # Index and middle fingers' tips must be highest parts of those fingers
        (index_tip.y < index_mcp.y) and (middle_tip.y < middle_mcp.y)
        and
        # Index finger's tips must be higher than other fingers' mcp
        (index_tip.y < ring_mcp.y) and (index_tip.y < pinky_mcp.y)
        and
        # Middle finger's tips must be higher than other fingers' mcp
        (middle_tip.y < ring_mcp.y) and (middle_tip.y < pinky_mcp.y)
    ):
        return True
    return False


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame and detect hands
    result = hands.process(frame_rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Draw hand landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Detect custom gesture (peace sign)
            if detect_peace_sign(hand_landmarks):
                cv2.putText(frame, 'Peace Sign Detected!', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                # Trigger interaction (e.g., print a message)
                print("Peace sign gesture detected!")

    # Display the frame
    cv2.imshow('MediaPipe Hands', frame)

    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

### Gesture Detection Functions
After checking how the upper ones worked, we implemented fixed, more robust versions of gesture detection functions.

In [47]:
import time

# Global variables for tracking gestures and cooldown

# The currently detected gesture
current_gesture = None

# Time when the current gesture expires
gesture_reset_time = 0

# Cooldown variables for scrolling
previous_thumb_tip = None
previous_index_tip = None
last_gesture_time = 0
COOLDOWN_PERIOD = 1.5

# Reset current gesture when it expires
def reset_gesture():
    global current_gesture, gesture_reset_time
    if time.time() > gesture_reset_time:
        current_gesture = None

# Gesture detection functions
def detect_peace_sign(hand_landmarks):
    
    # Track the time of the last detected gesture
    global last_gesture_time

    # Get landmarks
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]

    index_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
    middle_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]
    ring_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP]
    pinky_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP]

    # Check that index and middle are raised above their MCPs and other MCPs
    index_and_middle_up = (
        (index_tip.y < index_mcp.y) and
        (middle_tip.y < middle_mcp.y) and
        (index_tip.y < ring_mcp.y) and 
        (index_tip.y < pinky_mcp.y) and
        (middle_tip.y < ring_mcp.y) and 
        (middle_tip.y < pinky_mcp.y)
    )

    # Check spacing between index and middle fingers
    index_middle_spacing = abs(index_tip.x - middle_tip.x) > 0.1

    # Check that ring and pinky are down (their tips should be below their MCP joints)
    ring_and_pinky_down = (
        (ring_tip.y > ring_mcp.y + 0.02) and
        (pinky_tip.y > pinky_mcp.y + 0.02)
    )

    if index_and_middle_up and index_middle_spacing and ring_and_pinky_down:
        
        # Update last gesture time (to apply cooldown for peace sign gesture)
        last_gesture_time = time.time()
        return True
    return False



def detect_thumbs_up(hand_landmarks, margin=0.05):
    
    # Track the time of the last detected gesture
    global last_gesture_time 

    # Get landmarks
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]

    index_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
    middle_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]
    ring_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP]
    pinky_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP]
    thumb_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP]
    
    thumb_base = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_CMC]

    # Thumb tip should be above other fingertips
    thumb_tip_up = ((thumb_tip.y + margin < index_tip.y) and
                    (thumb_tip.y + margin < middle_tip.y) and
                    (thumb_tip.y + margin < ring_tip.y) and
                    (thumb_tip.y + margin < pinky_tip.y) and
                    (thumb_tip.y < thumb_mcp.y))
    
    # Other fingers should be in order from top to bottom
    other_fingers_ordered = ((index_mcp.y < middle_mcp.y) and
                             (middle_mcp.y < ring_mcp.y) and
                             (ring_mcp.y < pinky_mcp.y))
    
    
    if thumb_tip_up and other_fingers_ordered:
        
        # Update last gesture time (to apply cooldown for thumbs-up gesture)
        last_gesture_time = time.time()
        return True
    return False


def detect_thumbs_down(hand_landmarks, margin=0.05):
    
    # Track the time of the last detected gesture
    global last_gesture_time

    # Get landmarks
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]

    index_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
    middle_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]
    ring_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP]
    pinky_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP]

    # Thumb tip must be lower than anything else
    thumb_tip_down = ((thumb_tip.y > index_tip.y + margin) and
                    (thumb_tip.y > middle_tip.y + margin) and
                    (thumb_tip.y > ring_tip.y + margin) and
                    (thumb_tip.y > pinky_tip.y + margin))
    
    # Other fingers should be in the order (from top to down) pinky > ring > middle > index finger
    other_fingers_ordered = ((index_mcp.y > middle_mcp.y) and
                             (middle_mcp.y > ring_mcp.y) and
                             (ring_mcp.y > pinky_mcp.y))
    
    if thumb_tip_down and other_fingers_ordered:
        
        # Update last gesture time (to apply cooldown for thumbs-down gesture)
        last_gesture_time = time.time()
        return True
    return False

def detect_scroll(hand_landmarks, threshold=0.1, dominance_ratio=4.0):
    
    # Get previous positions and last gesture time
    global previous_thumb_tip, previous_index_tip, last_gesture_time

    # Get current positions
    current_thumb_tip_x = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x
    current_thumb_tip_y = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y
    current_index_tip_x = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x
    current_index_tip_y = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y

    # Check for cooldown
    current_time = time.time()
    
    # If last gesture happened not before cooldown, return False (no scroll)
    if current_time - last_gesture_time < COOLDOWN_PERIOD:
        return False, None 

    # Initialize previous positions if not set
    if previous_thumb_tip is None or previous_index_tip is None:
        previous_thumb_tip = (current_thumb_tip_x, current_thumb_tip_y)
        previous_index_tip = (current_index_tip_x, current_index_tip_y)
        return False, None

    # Calculate location changes
    thumb_horizontal_disp = current_thumb_tip_x - previous_thumb_tip[0]
    thumb_vertical_disp = current_thumb_tip_y - previous_thumb_tip[1]
    index_horizontal_disp = current_index_tip_x - previous_index_tip[0]
    index_vertical_disp = current_index_tip_y - previous_index_tip[1]

    # Average the movements of thumb and index for robustness
    horizontal_disp = (thumb_horizontal_disp + index_horizontal_disp) / 2
    vertical_disp = (thumb_vertical_disp + index_vertical_disp) / 2

    # Determine dominant movement (we want to return only a horizontal or vertical movement)
    horizontal_movement = abs(horizontal_disp) > threshold
    vertical_movement = abs(vertical_disp) > threshold

    # Check dominance
    if horizontal_movement and abs(horizontal_disp) > dominance_ratio * abs(vertical_disp):
        direction = "right" if horizontal_disp > 0 else "left"
        dominant_axis = "horizontal"
    elif vertical_movement and abs(vertical_disp) > dominance_ratio * abs(horizontal_disp):
        direction = "down" if vertical_disp > 0 else "up"
        dominant_axis = "vertical"
    else:
        direction = None
        dominant_axis = None

    # If there is a dominant movement
    if dominant_axis:
        
        # Update positions
        previous_thumb_tip = (current_thumb_tip_x, current_thumb_tip_y)
        previous_index_tip = (current_index_tip_x, current_index_tip_y)
        
        # Update last gesture time
        last_gesture_time = current_time 
        return True, direction

    return False, None

In [48]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)

# Global variables for tracking previous positions
previous_thumb_tip = None
previous_index_tip = None

print("Press 'Esc' to exit.")

# Initialize gesture display variables
current_gesture = None 
gesture_display_time = 0 
GESTURE_DISPLAY_DURATION = 1.5 

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame and detect hands
    result = hands.process(frame_rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Draw hand landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Detect gestures
            detected_gesture = None
            if detect_peace_sign(hand_landmarks):
                detected_gesture = "Peace Sign Detected!"
            elif detect_thumbs_up(hand_landmarks):
                detected_gesture = "Thumbs Up Detected!"
            elif detect_thumbs_down(hand_landmarks):
                detected_gesture = "Thumbs Down Detected!"
            
            # If a gesture is detected, update the display variables
            if detected_gesture:
                current_gesture = detected_gesture
                gesture_display_time = time.time()
            else:
                # Only check for scroll if no other gesture is detected
                detected, direction = detect_scroll(hand_landmarks, threshold=0.1, dominance_ratio=4.0)
                if detected:
                    current_gesture = f"Scroll Detected: {direction.title()}!"
                    gesture_display_time = time.time()

    # Display the current gesture if within the display duration
    if current_gesture and (time.time() - gesture_display_time < GESTURE_DISPLAY_DURATION):
        cv2.putText(frame, current_gesture, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the frame
    cv2.imshow('Gesture Recognition', frame)

    # Break the loop when 'Esc' key is pressed
    if cv2.waitKey(5) & 0xFF == 27:
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()

I0000 00:00:1733588299.302138  653089 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1
W0000 00:00:1733588299.310918  697430 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733588299.317616  697432 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Press 'Esc' to exit.


### Cursor control

In [17]:
import cv2
import mediapipe as mp
import pyautogui

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Get screen dimensions
screen_width, screen_height = pyautogui.size()

def is_click_gesture(hand_landmarks):
    """Detect a pinching gesture for a left click."""
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]

    # Calculate the 3D Euclidean distance between index tip and thumb tip
    distance = ((index_tip.x - thumb_tip.x) ** 2 +
                (index_tip.y - thumb_tip.y) ** 2 +
                (index_tip.z - thumb_tip.z) ** 2) ** 0.5

    # Adjust threshold based on typical 3D distances observed
    return distance < 0.05


# Open webcam
cap = cv2.VideoCapture(0)

print("Press 'Esc' to exit.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally for a mirror-like effect
    frame = cv2.flip(frame, 1)

    # Get frame dimensions
    frame_height, frame_width, _ = frame.shape

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame and detect hands
    result = hands.process(frame_rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Draw hand landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get index finger tip coordinates
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

            # Normalize coordinates to screen dimensions
            cursor_x = int(index_finger_tip.x * screen_width)
            cursor_y = int(index_finger_tip.y * screen_height)

            # Move the mouse cursor
            pyautogui.moveTo(cursor_x, cursor_y)

            # Detect click gesture
            if is_click_gesture(hand_landmarks):
                pyautogui.click()
                cv2.putText(frame, "Click!", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow("Gesture-Based Cursor Control", frame)

    # Break the loop when 'Esc' key is pressed
    if cv2.waitKey(5) & 0xFF == 27:
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()

I0000 00:00:1733577027.043797  567238 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1
W0000 00:00:1733577027.055785  601304 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733577027.062216  601304 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Press 'Esc' to exit.
